In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

In [4]:
image_info = pd.read_csv('../processed_data/ISIC_2019_Training_GroundTruth_Processed_Balanced.csv')

In [5]:
image_info

,image,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK,Cat,y
0,ISIC_0000001,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NV,0
1,ISIC_0000002,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,MEL,1
2,ISIC_0000003,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NV,0
3,ISIC_0000004,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,MEL,1
4,ISIC_0000007,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NV,0
...,...,...,...,...,...,...,...,...,...,...,...,...
24801,ISIC_0073153_flipped,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,AK,1
24802,ISIC_0073157_flipped,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,AK,1
24803,ISIC_0073198_flipped,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,AK,1
24804,ISIC_0073214_flipped,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,AK,1


In [6]:
def get_image(df):
    return np.asarray(Image.open((f"../processed_data/{df['Cat']}/{df['image']}.jpg")))

In [8]:
image_info.shape

(24806, 12)

In [12]:
X[0].shape

(400, 400, 3)

In [17]:
X = image_info[:1000].apply(get_image, axis=1)
X

0      [[[204, 204, 214], [204, 204, 214], [204, 204,...
1      [[[135, 146, 178], [135, 146, 178], [136, 147,...
2      [[[235, 235, 237], [235, 235, 237], [236, 236,...
3      [[[115, 115, 115], [115, 115, 115], [115, 115,...
4      [[[126, 135, 144], [128, 137, 146], [130, 138,...
                             ...                        
995    [[[188, 144, 117], [187, 143, 116], [188, 144,...
996    [[[205, 178, 171], [206, 179, 172], [206, 179,...
997    [[[199, 163, 149], [206, 170, 156], [212, 176,...
998    [[[157, 127, 125], [157, 129, 126], [159, 131,...
999    [[[144, 116, 102], [144, 116, 102], [144, 116,...
Length: 1000, dtype: object

In [20]:
y = image_info['y'][:1000]
y

0      0
1      1
2      0
3      1
4      0
      ..
995    1
996    1
997    1
998    1
999    1
Name: y, Length: 1000, dtype: int64

In [18]:
from sklearn.model_selection import train_test_split   

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)